In [1]:
import requests
import json
from urllib.parse import urlencode, quote_plus
import random
import pandas as pd
from time import sleep
from tqdm import tqdm

StockX internal API:

```
https://stockx.com/api/products/5b85a42d-daed-44a8-b75e-42ec85fe41af/activity?state=480&currency=USD&limit=10&page=1&sort=createdAt&order=DESC&country=RU
```

In [2]:
def get_item_uuids(data_path='../data pages.csv'):
    return list(set(pd.read_csv(data_path)['uuid'].tolist()))

In [3]:
BASE_API_URL = 'https://stockx.com/api/products/'
DATA_LOC = '/activity?'

params = {
    'state': 480,
    'currency': 'USD'
    # 'limit': 10,
    # 'page': 1
    # 'sort': 'createdAt',
    # 'order': 'DESC',
    # 'country': 'RU'
}

with open('./user-agents.json', 'r', encoding='utf-8') as f:
    headers = json.load(f)

def get_sales_data_for_item(uuid):
    sleep_time = 5
    while True:
        resp = requests.get(BASE_API_URL + uuid + DATA_LOC +
                                urlencode(params, quote_via=quote_plus),
                            headers=random.choice(headers))
        
        if resp.status_code == 200:
            return resp.json()
        
        sleep(sleep_time)
        sleep_time *= 2

In [4]:
def get_all_sales_data(uuids: list):
    with open('./saved/done.json', 'r') as f:
        done = json.load(f)
    uuids = [ uuid for uuid in uuids if uuid not in done ]
    
    for uuid in tqdm(uuids, ncols=100):
        parsed_data = get_sales_data_for_item(uuid)
        temp_df = pd.DataFrame(parsed_data)
        temp_df.to_csv(f'./saved/{uuid}.csv')
        done.append(uuid)
        
        with open('./saved/done.json', 'w') as f:
            json.dump(done, f)
        
        sleep(5)
    
    print('All done')

In [5]:
items = get_item_uuids('./data pages.csv')
get_all_sales_data(items)

  0%|                                                          | 1/20878 [00:06<36:59:27,  6.38s/it]


KeyboardInterrupt: 